In [3]:
#json檔處理
'''csv檔為簡單的二維表格資料；而json檔(文字檔)為一複雜的資料結構，裡面可儲存的資料非常多。
因其複雜性極高，故需要先用一json架構的解析器來了解其架構(上網搜尋online json viewer即可)以便寫程式之所需。'''

##下載文字檔
import requests

url = "https://opendata.cwa.gov.tw/api/v1/rest/datastore/F-C0032-001?Authorization=rdec-key-123-45678-011121314&format=JSON"
response = requests.get(url, verify = False)  #加入參數verify = False使其不去驗證SSL憑證
if response.ok:  #a property that returns True if status_code is less than 400, False if not
    print("下載成功")
    json_str:str = response.text  #.text: content of the response, unicode
else:
    print("下載失敗")

下載成功


/home/vscode/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'opendata.cwa.gov.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [4]:
##將json文字資料轉為python資料結構以便後續使用-方法一
'''json為一內建module，可用於JSON的編碼(json.dumps())以及解碼(json.loads())。
注意：在使用之前記得要先匯入模組。'''

import json

jsonData = json.loads(json_str)  
#json.loads(s)為一function，參數s可為含json文檔的str, bytes or bytearray instance，此函數可將s依轉換表(如將json中的string解碼為python中的str)反序列化為python對象/實體
jsonData

{'success': 'true',
 'result': {'resource_id': 'F-C0032-001',
  'fields': [{'id': 'datasetDescription', 'type': 'String'},
   {'id': 'locationName', 'type': 'String'},
   {'id': 'parameterName', 'type': 'String'},
   {'id': 'parameterValue', 'type': 'String'},
   {'id': 'parameterUnit', 'type': 'String'},
   {'id': 'startTime', 'type': 'Timestamp'},
   {'id': 'endTime', 'type': 'Timestamp'}]},
 'records': {'datasetDescription': '三十六小時天氣預報',
  'location': [{'locationName': '嘉義縣',
    'weatherElement': [{'elementName': 'Wx',
      'time': [{'startTime': '2024-05-17 18:00:00',
        'endTime': '2024-05-18 06:00:00',
        'parameter': {'parameterName': '多雲', 'parameterValue': '4'}},
       {'startTime': '2024-05-18 06:00:00',
        'endTime': '2024-05-18 18:00:00',
        'parameter': {'parameterName': '多雲時陰', 'parameterValue': '5'}},
       {'startTime': '2024-05-18 18:00:00',
        'endTime': '2024-05-19 06:00:00',
        'parameter': {'parameterName': '多雲時陰', 'parameterValue'

In [5]:
##將json文字資料轉為python資料結構以便後續使用(不須先下載為文字檔)-方法二
'''以上兩個程式區塊的作法是先將json的文字檔下載下來，然後再將這些文字資料轉換為python的資料結構。
我們也可透過Response的實體方法直接將json檔轉換為python的資料結構，如此便可少掉下載文字檔的動作。'''

import json

url = "https://opendata.cwa.gov.tw/api/v1/rest/datastore/F-C0032-001?Authorization=rdec-key-123-45678-011121314&format=JSON"
response = requests.get(url,verify = False)
jsonData1 = response.json()  #.json(*): a method in requests.Response, returns the json-encoded content of a response, if any
print(jsonData1["success"])  #true
print(jsonData1["records"]["datasetDescription"])  #三十六小時天氣預報
jsonData1


true
三十六小時天氣預報


/home/vscode/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'opendata.cwa.gov.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'success': 'true',
 'result': {'resource_id': 'F-C0032-001',
  'fields': [{'id': 'datasetDescription', 'type': 'String'},
   {'id': 'locationName', 'type': 'String'},
   {'id': 'parameterName', 'type': 'String'},
   {'id': 'parameterValue', 'type': 'String'},
   {'id': 'parameterUnit', 'type': 'String'},
   {'id': 'startTime', 'type': 'Timestamp'},
   {'id': 'endTime', 'type': 'Timestamp'}]},
 'records': {'datasetDescription': '三十六小時天氣預報',
  'location': [{'locationName': '嘉義縣',
    'weatherElement': [{'elementName': 'Wx',
      'time': [{'startTime': '2024-05-17 18:00:00',
        'endTime': '2024-05-18 06:00:00',
        'parameter': {'parameterName': '多雲', 'parameterValue': '4'}},
       {'startTime': '2024-05-18 06:00:00',
        'endTime': '2024-05-18 18:00:00',
        'parameter': {'parameterName': '多雲時陰', 'parameterValue': '5'}},
       {'startTime': '2024-05-18 18:00:00',
        'endTime': '2024-05-19 06:00:00',
        'parameter': {'parameterName': '多雲時陰', 'parameterValue'

In [10]:
##用response實體方法將json檔直接轉換為python的資料結構

import json

jsonRecords = jsonData1['records']['location']
print(type(jsonRecords))
for item in jsonRecords:
    print(item)

<class 'list'>
{'locationName': '嘉義縣', 'weatherElement': [{'elementName': 'Wx', 'time': [{'startTime': '2024-05-17 18:00:00', 'endTime': '2024-05-18 06:00:00', 'parameter': {'parameterName': '多雲', 'parameterValue': '4'}}, {'startTime': '2024-05-18 06:00:00', 'endTime': '2024-05-18 18:00:00', 'parameter': {'parameterName': '多雲時陰', 'parameterValue': '5'}}, {'startTime': '2024-05-18 18:00:00', 'endTime': '2024-05-19 06:00:00', 'parameter': {'parameterName': '多雲時陰', 'parameterValue': '5'}}]}, {'elementName': 'PoP', 'time': [{'startTime': '2024-05-17 18:00:00', 'endTime': '2024-05-18 06:00:00', 'parameter': {'parameterName': '0', 'parameterUnit': '百分比'}}, {'startTime': '2024-05-18 06:00:00', 'endTime': '2024-05-18 18:00:00', 'parameter': {'parameterName': '10', 'parameterUnit': '百分比'}}, {'startTime': '2024-05-18 18:00:00', 'endTime': '2024-05-19 06:00:00', 'parameter': {'parameterName': '10', 'parameterUnit': '百分比'}}]}, {'elementName': 'MinT', 'time': [{'startTime': '2024-05-17 18:00:00', '

In [12]:
##抓取特定資料並用DataFrame顯示

import json
import pandas as pd

jsonRecords = jsonData1['records']['location']
weather_list:list[dict] = []  #重新規劃資料：建立一新list來存放dict資料
for item in jsonRecords:
    city_item:dict = {}  #建立一新dict來抓取資料值
    city_item["城市"] = item["locationName"]  #注意：此處是用迴圈抓值，即item本身就有索引值的用意，故此處可不用寫多寫[0]，直接寫item["locationName"]即可
    city_item["起始時間"] = item["weatherElement"][0]["time"][0]["startTime"]
    city_item["結束時間"] = item["weatherElement"][0]["time"][0]["endTime"]
    city_item["最低溫度"] = float(item["weatherElement"][2]["time"][0]["parameter"]["parameterName"])  #將str轉型別為float
    city_item["最高溫度"] = float(item["weatherElement"][4]["time"][0]["parameter"]["parameterName"])
    city_item["感覺"] = item["weatherElement"][3]["time"][0]["parameter"]["parameterName"]
    weather_list.append(city_item)
    # weather_list.append({"城市":item["locationName"]})
df = pd.DataFrame(weather_list)
    
# weather_list
df

,城市,起始時間,結束時間,最低溫度,最高溫度,感覺
0,嘉義縣,2024-05-17 18:00:00,2024-05-18 06:00:00,22.0,27.0,舒適
1,新北市,2024-05-17 18:00:00,2024-05-18 06:00:00,23.0,27.0,舒適
2,嘉義市,2024-05-17 18:00:00,2024-05-18 06:00:00,22.0,28.0,舒適
3,新竹縣,2024-05-17 18:00:00,2024-05-18 06:00:00,22.0,25.0,舒適
4,新竹市,2024-05-17 18:00:00,2024-05-18 06:00:00,22.0,25.0,舒適
5,臺北市,2024-05-17 18:00:00,2024-05-18 06:00:00,22.0,27.0,舒適
6,臺南市,2024-05-17 18:00:00,2024-05-18 06:00:00,24.0,28.0,舒適
7,宜蘭縣,2024-05-17 18:00:00,2024-05-18 06:00:00,21.0,25.0,舒適
8,苗栗縣,2024-05-17 18:00:00,2024-05-18 06:00:00,20.0,25.0,舒適
9,雲林縣,2024-05-17 18:00:00,2024-05-18 06:00:00,21.0,27.0,舒適


In [13]:
##將DataFrame儲存為csv檔/excel檔
'''pd.DataFrame.to_csv(): write object to a comma-separated values (csv) file. 為一DataFrame的method；
pd.DataFrame.to_excel(excel_writer): write object to an execel sheet. 為一DataFrame的method，
其中參數excel_writer可為excel檔案路徑或現有的pd.ExcelWriter物件(path-like, file-like, or ExcelWriter object)。
注意：要先建好DataFrame，方能使用這些method。'''

import pandas as pd

df.to_csv("天氣預測1天.csv")  #注意：記得要給副檔名
df.to_excel("天氣預測1天.xlsx")  #df為檔案路徑

In [3]:
#作業
import requests
import json
import pandas as pd

url ='https://opendata.cwa.gov.tw/api/v1/rest/datastore/O-A0001-001?Authorization=rdec-key-123-45678-011121314&format=JSON&WeatherElement=Weather'
jsonData = requests.get(url,verify = False).json()
jsonRecords = jsonData['records']["Station"]
station_list:list[dict] = []
for item in jsonRecords:
    station_item:dict = {}
    station_item["日期時間"] = item["ObsTime"]["DateTime"]
    station_item["縣市"] = item["GeoInfo"]["CountyName"]
    station_item["區域"] = item["GeoInfo"]["TownName"]
    station_item["海拔"] = item["GeoInfo"]["StationAltitude"]
    station_item["狀態"] = item["WeatherElement"]["Weather"]
    station_list.append(station_item)
df = pd.DataFrame(station_list)
    
df

/home/vscode/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'opendata.cwa.gov.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,日期時間,縣市,區域,海拔,狀態
0,2024-01-04T19:00:00+08:00,屏東縣,滿州鄉,31.0,多雲
1,2024-01-04T19:00:00+08:00,屏東縣,滿州鄉,13.0,陰
2,2024-01-04T19:00:00+08:00,屏東縣,恆春鎮,13.0,陰
3,2024-01-04T19:00:00+08:00,屏東縣,恆春鎮,33.0,陰
4,2024-01-04T19:00:00+08:00,屏東縣,車城鄉,30.0,陰
...,...,...,...,...,...
464,2024-01-04T19:00:00+08:00,嘉義縣,阿里山鄉,440.0,多雲
465,2024-01-04T19:00:00+08:00,雲林縣,古坑鄉,117.0,晴
466,2024-01-04T19:00:00+08:00,雲林縣,水林鄉,15.0,多雲
467,2024-01-04T19:00:00+08:00,臺南市,北門區,10.0,多雲
